In [1]:
###import comstockpostproc-standard things, and then don't use most of them

import os
from textwrap import indent

import boto3
import logging
import numpy as np
import pandas as pd

from comstockpostproc.resstock_naming_mixin_LARGEE import ResStockNamingMixin
from comstockpostproc.units_mixin import UnitsMixin
from comstockpostproc.s3_utilities_mixin import S3UtilitiesMixin
from comstockpostproc import resstock_LARGEE

logger = logging.getLogger(__name__)

In [44]:
class ResStock_data_process():
    def __init__(self, resstock_results_folder, resstock_file_name, downselect_rows_tf, downselect_row_fields, 
                 values_to_keep,  col_plan_folder, col_plan_name,
                 add_wide_fields_tf, dfs_for_wide_fields, wide_mergeon_fields, wide_merge_cols, wide_col_plans, wide_merge_newnames,
                 add_local_bills_tf, downselect_cols_tf, add_long_fields_tf, rate_inputs_df, 
                 long_fields_also_wide_tf, long_fields_also_wide, long_fields_also_wide_names,
                 output_folder, output_file_name):
        """
        A class to load and transform ResStock 2024.2 data for futher steps in an automated workflow
        """
        #initialize members
        self.resstock_results_folder = resstock_results_folder
        self.resstock_file_name = resstock_file_name
        self.downselect_rows_tf = downselect_rows_tf
        self.downselect_row_fields = downselect_row_fields
        self.values_to_keep = values_to_keep
        self.col_plan_folder = col_plan_folder
        self.col_plan_name = col_plan_name
        self.add_wide_fields_tf = add_wide_fields_tf
        self.dfs_for_wide_fields = dfs_for_wide_fields
        self.wide_mergeon_fields = wide_mergeon_fields
        self.wide_merge_cols = wide_merge_cols
        self.wide_col_plans = wide_col_plans
        self.wide_merge_newnames = wide_merge_newnames
        self.add_local_bills_tf = add_local_bills_tf
        self.downselect_cols_tf = downselect_cols_tf
        self.add_long_fields_tf = add_long_fields_tf
        self.rate_inputs_df = rate_inputs_df
        self.long_fields_also_wide_tf = long_fields_also_wide_tf
        self.long_fields_also_wide = long_fields_also_wide
        self.long_fields_also_wide_names = long_fields_also_wide_names
        self.output_folder = output_folder
        self.output_file_name = output_file_name


        #execute
        self.download_data()
        self.downselect_rows()
        self.make_col_plan()
        self.add_wide_fields()
        self.downselect_cols()
        self.pivot_data()
        self.add_long_fields()
        self.categorize_outputs()
        self.addl_wide_fields_in_long()
        self.add_weighted_values_col()
        self.save_file()

    def download_data(self):
        #print (1)
    #load results from already-downloaded OEDI file
        results_file_path = os.path.join(self.resstock_results_folder, self.resstock_file_name)
        self.data = pd.read_csv(results_file_path, engine = "pyarrow")

    def downselect_rows(self):
        #print (2)
    #downselect to a subset of results 
        if(self.downselect_rows_tf == True):
            for field, values in zip(self.downselect_row_fields, self.values_to_keep):
                self.data = self.data.loc[self.data[field].isin(values)]

    def make_col_plan(self):
        #print (3)
    #assign a plan for each column in the dataset, from a premade csv
        plan_file_path = os.path.join(self.col_plan_folder, self.col_plan_name)
        self.col_plan = pd.read_csv(plan_file_path, engine = "pyarrow")
        #create lists of columns
        self.cols_to_remove = self.col_plan.loc[self.col_plan['plan']=='remove', 'column'].tolist()
        self.cols_wide = self.col_plan.loc[self.col_plan['plan']=='keep', 'column'].tolist()
        self.cols_to_pivot = self.col_plan.loc[self.col_plan['plan']=='pivot', 'column'].tolist()

    def add_wide_fields(self):
        #print (4)
    #add additional wide format fields before pivoting, and also add plans for them
        #add additional precomputed wide format fields before pivoting
        if(self.add_wide_fields_tf == True):
            for dfw, wide_mergeon_field, wide_merge_col, wide_merge_newname, wide_col_plan in zip(
                self.dfs_for_wide_fields, self.wide_mergeon_fields, self.wide_merge_cols, self.wide_merge_newnames, self.wide_col_plans):
                self.data = self.data.merge(dfw, [[wide_mergeon_field, wide_merge_col]], on = wide_mergeon_field, how = "left")
                self.data.rename(columns = {wide_merge_col:wide_merge_newname}, inplace = True)
                if self.wide_col_plan == 'pivot':
                    self.cols_to_pivot = self.cols_to_pivot + [wide_merge_newname]
                elif self.wide_col_plan == 'keep':
                    self.cols_wide = self.cols_wide + [wide_merge_newname]
                else:
                    self.cols_to_remove = self.cols_to_remove + [wide_merge_newname]
        #add local bills before pivoting
        if(self.add_local_bills_tf == True):
            for index, row in self.rate_inputs_df.iterrows():
                self.data[row['column']] = row['fixed monthly cost']*12 + row['variable cost per kwh']*(self.data[row['col list for scaling']].sum(axis = 1))
                if row['plan'] == 'pivot':
                    self.cols_to_pivot = self.cols_to_pivot + [row['column']]
                elif row['plan'] == 'keep':
                    self.cols_wide = self.cols_wide + [row['column']]
                else:
                    self.cols_to_remove = self.cols_to_remove + [row['column']]
            plan_for_new_cols_df = self.rate_inputs_df.drop(['fixed monthly cost', 'variable cost per kwh', 'col list for scaling'], axis = 1)
            self.col_plan = pd.concat([self.col_plan, plan_for_new_cols_df], axis = 0)
    
    def downselect_cols(self):
        #print (5)
    #remove unneceessary columns
        if(self.downselect_cols_tf == True):
            self.data.drop(self.data[self.cols_to_remove], axis = 1, inplace = True)
            #print(self.data.columns)

    def pivot_data(self):
        #print (6)
    #make all the results long format, keep the characteristics wide
        self.data_long = pd.melt(
            self.data,
            id_vars = self.cols_wide,
            var_name = "Output",
            value_name = "Value"
        )
    
    def add_long_fields(self):
        #print (7)
    #this is where you add any long format fields.
        if(self.add_long_fields_tf == True):
            print('7a')

    def categorize_outputs(self):
        #print (8)
    #Develop output categorization
        #split original column name
        self.data_long[['out', 'Result', 'End Use', 'Type', 'Units']] = self.data_long['Output'].str.split('.', expand = True)
        #only keep the units
        self.data_long.drop(self.data_long[['out', 'Result', 'End Use', 'Type']], axis = 1, inplace = True)
        #use mappings to get the categorizations
        out_cats = self.col_plan.drop(self.col_plan[["col_type", "plan"]], axis = 1, inplace = False)
        self.data_long = self.data_long.merge(out_cats, left_on = 'Output', right_on = "column", how = 'left')

    def addl_wide_fields_in_long(self):
        #print (9)
    #re-merge in any long fields that are also needed as wide fields
        if(self.long_fields_also_wide_tf == True):
            merge_data_cols = ["bldg_id"] + self.long_fields_also_wide
            self.data_long = self.data_long.merge(self.data[merge_data_cols], on = "bldg_id", how = "left")
            for colname, newcolname in zip(self.long_fields_also_wide, self.long_fields_also_wide_names):
                self.data_long.rename(columns = {colname:newcolname}, inplace = True)

    def add_weighted_values_col(self):
        #print(10)
        self.data_long['Weighted Value'] = self.data_long['Value']*self.data_long['weight']
    
    def save_file(self):
       # print(11)
    #save file
        self.data_long.to_csv(os.path.join(self.output_folder, self.output_file_name))

In [45]:
####Prepare utility rates for C2C DV

##project-specific utility bills - inputs
#Electricity
fixed_elec_cost_monthly = 10.56
var_elec_cost_per_kwh = 0.17404 #cf 0.137/kwh

#Natural Gas
fixed_ng_cost_monthly = 16.25
var_ng_cost_per_ccf = 1.495

#Fuel Oil
var_fo_cost_per_gal = 2.851

#Propane
var_propane_cost_per_gal = 3.199

##project-specific utility bills - unit conversions
gal_fuel_oil_to_mbtu = 139/1000
gal_propane_to_mbtu = 91.6 / 1000
mbtu_to_kwh = 293.0710701722222
dol_per_ccf_to_dol_per_therm = 1/1.038 #$ per Ccf divided by 1.038 equals $ per therm https://www.eia.gov/tools/faqs/faq.php?id=45&t=8
therm_to_kwh = 29.307107017222222

var_ng_cost_per_kwh = var_ng_cost_per_ccf * (dol_per_ccf_to_dol_per_therm) * (1/therm_to_kwh)
var_fo_cost_per_kwh = var_fo_cost_per_gal * (1/gal_fuel_oil_to_mbtu) * (1/mbtu_to_kwh)
var_propane_cost_per_kwh = var_propane_cost_per_gal * (1/gal_propane_to_mbtu) * (1/mbtu_to_kwh)

#print(var_ng_cost_per_kwh) #0.011423113848862812, cf 0.0339307/kwh
#print(var_fo_cost_per_kwh) #0.06998572515142275, cf 0.0704125/kwh
#print(var_propane_cost_per_kwh) #0.11916420397790706. cf 0.101456/kWh

#assemble for input
rates_data_inputs = [
    ["out.bills_local.electricity.total.usd", fixed_elec_cost_monthly, var_elec_cost_per_kwh, ["out.electricity.total.energy_consumption.kwh"], "out.x", "pivot", "Utility Bills", "Electricity", "Electricity Total", "Total"],
    ["out.bills_local.natural_gas.total.usd", fixed_ng_cost_monthly, var_ng_cost_per_kwh, ["out.natural_gas.total.energy_consumption.kwh"], "out.x", "pivot", "Utility Bills", "Natural Gas", "Natural Gas Total", "Total"],
    ["out.bills_local.fuel_oil.total.usd", 0, var_fo_cost_per_kwh, ["out.fuel_oil.total.energy_consumption.kwh"], "out.x", "pivot", "Utility Bills", "Fuel Oil", "Fuel Oil Total", "Total"],
    ["out.bills_local.propane.total.usd", 0, var_propane_cost_per_kwh, ["out.propane.total.energy_consumption.kwh"], "out.x", "pivot", "Utility Bills", "Energy", "Total", "Total"],
    ["out.bills_local.all_fuels.total.usd", 0, 1, ["out.bills_local.electricity.total.usd", "out.bills_local.natural_gas.total.usd", "out.bills_local.fuel_oil.total.usd", "out.bills_local.propane.total.usd"], "out.x", "pivot", "Utility Bills", "Energy", "Total", "Total"]
]

rate_inputs_df = pd.DataFrame(rates_data_inputs, columns = ['column', 'fixed monthly cost', 'variable cost per kwh', 'col list for scaling', 'col_type', 'plan', 'Result Type', 'Fuel', 'End Use', 'End Use Category'])


In [46]:
ResStock_data_process(
    resstock_results_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Data/2024.2/AMY2018",
    resstock_file_name = "PA_baseline_metadata_and_annual_results.csv",
    downselect_rows_tf = True, 
    downselect_row_fields = ["in.county_name"],
    values_to_keep = [["Montgomery County", "Bucks County", "Chester County", "Delaware County"]],
    col_plan_folder = 'C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Baseline (4.2)',
    col_plan_name = 'Column Plan.csv',
    add_wide_fields_tf = False,
    dfs_for_wide_fields = 'NA',
    wide_mergeon_fields = 'NA',
    wide_merge_cols = 'NA', 
    wide_col_plans = 'NA',
    wide_merge_newnames = 'NA',
    add_local_bills_tf = True,
    downselect_cols_tf = True, 
    add_long_fields_tf = False, 
    rate_inputs_df = rate_inputs_df, 
    long_fields_also_wide_tf = True,
    long_fields_also_wide = ["out.emissions.all_fuels.lrmer_mid_case_15.co2e_kg"], #, "out.bills_local.all_fuels.total.usd"], 
    long_fields_also_wide_names = ["Emissions"], #, Utility Bills Total", "Emissions"],
    output_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Baseline (4.2)", 
    output_file_name = "resstock_function_test_1.csv"
    )

['bldg_id', 'upgrade', 'weight', 'applicability', 'in.sqft', 'in.representative_income', 'in.ahs_region', 'in.aiannh_area', 'in.area_median_income', 'in.ashrae_iecc_climate_zone_2004', 'in.ashrae_iecc_climate_zone_2004_2_a_split', 'in.bathroom_spot_vent_hour', 'in.battery', 'in.bedrooms', 'in.building_america_climate_zone', 'in.cec_climate_zone', 'in.ceiling_fan', 'in.census_division', 'in.census_division_recs', 'in.census_region', 'in.city', 'in.clothes_dryer', 'in.clothes_dryer_usage_level', 'in.clothes_washer', 'in.clothes_washer_presence', 'in.clothes_washer_usage_level', 'in.cooking_range', 'in.cooking_range_usage_level', 'in.cooling_setpoint', 'in.cooling_setpoint_has_offset', 'in.cooling_setpoint_offset_magnitude', 'in.cooling_setpoint_offset_period', 'in.corridor', 'in.county', 'in.county_and_puma', 'in.county_name', 'in.dehumidifier', 'in.dishwasher', 'in.dishwasher_usage_level', 'in.door_area', 'in.doors', 'in.duct_leakage_and_insulation', 'in.duct_location', 'in.eaves', 'in.

In [14]:
#for index, row in self.rate_inputs_df.iterrows():
#                self.data_long[row['column']] = row['fixed monthly cost']*12 + row['variable cost per kwh']*sum(data_long[row['col list for scaling']])

for index, row in rate_inputs_df.iterrows():
    sum()
    print (row['column'], row['fixed monthly cost']*12)

out.bills_local.electricity.total.usd 126.72
out.bills_local.natural_gas.total.usd 195.0
out.bills_local.fuel_oil.total.usd 0.0
out.bills_local.propane.total.usd 0.0
out.bills_local.all_fuels.total.usd 0.0
